### 基礎的な操作

ssh で 192.168.99.199 で接続してください(openstack/openstack)

root ユーザのホームディレクトリには OpenStack を管理者ユーザーで使うための設定ファイルが保存されています。

openstack ユーザのホームディレクトリには OpenStack を一般ユーザで使うための設定ファイルが保存されています。

#### 認証情報の読み込み（openstackユーザで実施しています）
openstackユーザのホームディレクトリの openrc には演習用環境を利用するためのOpenStack認証情報が記載されています。各種コマンドで操作を行う前に、このファイルを読み込んで、環境変数へ認証情報を設定しておきます。

In [1]:
cat ~/openrc

export OS_AUTH_URL=http://192.168.99.199:5000/v2.0
export OS_REGION_NAME=RegionOne
export OS_TENANT_NAME=project01
export OS_USERNAME=openstack
export OS_PASSWORD=openstack



In [2]:
source ~/openrc

In [3]:
env | grep OS_

OS_REGION_NAME=RegionOne
OS_PASSWORD=openstack
OS_AUTH_URL=http://192.168.99.199:5000/v2.0
OS_USERNAME=openstack
OS_TENANT_NAME=project01


#### 初期状態の確認（一般ユーザーの初期状態）
初期状態では、管理者が設定する public ネットワーク、Cirrosイメージ作成されています。
みなさんがユーザとしてOpenStackを使う場合は、この状態からのスタートになります。

VMはありません

In [4]:
nova list

+----+------+--------+------------+-------------+----------+
| ID | Name | Status | Task State | Power State | Networks |
+----+------+--------+------------+-------------+----------+
+----+------+--------+------------+-------------+----------+


管理者が作成した public ネットワークのみが存在しています。

In [5]:
neutron net-list

+----------------------------------+--------+----------------------------------+
| id                               | name   | subnets                          |
+----------------------------------+--------+----------------------------------+
| 17e4bba8-2a15-46fb-              | public | a271b0a1-c912-492c-858e-         |
| ab54-fb463aee503c                |        | d5bd41888246                     |
+----------------------------------+--------+----------------------------------+


今回の演習用に作成したキーペアが作成されています。

In [6]:
nova keypair-list

+--------+------+-------------------------------------------------+
| Name   | Type | Fingerprint                                     |
+--------+------+-------------------------------------------------+
| my-key | ssh  | 43:0f:f5:b3:3e:ab:b7:1b:6a:a9:fc:65:4a:e6:42:89 |
+--------+------+-------------------------------------------------+


テスト用のOSイメージが登録されています。

In [7]:
openstack image list

+--------------------------------------+--------------+--------+
| ID                                   | Name         | Status |
+--------------------------------------+--------------+--------+
| 6466641d-f5e5-4cf5-8fe0-549a6a8a45d7 | CentOS7      | active |
| 66722486-ac79-4cd2-9a7d-c2479a9f6f92 | cirros-0.3.5 | active |
+--------------------------------------+--------------+--------+


フレーバー（AWSのインスタンスタイプ）が予めいくつか定義されています。これは管理者が設定します。

In [8]:
nova flavor-list

+----+----------+-----------+------+-----------+------+-------+-------------+-----------+
| ID | Name     | Memory_MB | Disk | Ephemeral | Swap | VCPUs | RXTX_Factor | Is_Public |
+----+----------+-----------+------+-----------+------+-------+-------------+-----------+
| 1  | m1.tiny  | 512       | 1    | 0         |      | 1     | 1.0         | True      |
| 2  | m1.small | 1024      | 10   | 0         |      | 1     | 1.0         | True      |
+----+----------+-----------+------+-----------+------+-------+-------------+-----------+


管理的なファイアーウォール機能が予め２つ定義されています。default は初期状態で存在し、すべての通信（→VM）を遮断します。ここでは open-all という全ての通信を許可するルールを演習用に作成しています。

In [9]:
neutron security-group-list

+---------------------------------+----------+---------------------------------+
| id                              | name     | security_group_rules            |
+---------------------------------+----------+---------------------------------+
| 43a9ef6f-2cf2-484f-a37a-        | default  | egress, IPv4                    |
| d45a290ac5db                    |          | egress, IPv6                    |
|                                 |          | ingress, IPv4, remote_group_id: |
|                                 |          | 43a9ef6f-2cf2-484f-a37a-        |
|                                 |          | d45a290ac5db                    |
|                                 |          | ingress, IPv6, remote_group_id: |
|                                 |          | 43a9ef6f-2cf2-484f-a37a-        |
|                                 |          | d45a290ac5db                    |
| b366a40b-4a47-4fbc-             | open-all | egress, IPv4                    |
| af51-4e214a29530d         

### 基本的な操作
操作の流れ。
* 仮想ルーターの作成 → 仮想ルーターと外部ネットワークを接続する（しなくても構いません。その場合は内部ネットワーク同士のルーティングに使います）
* 仮想ネットワークの作成 → 仮想サブネットの作成
* 仮想ルーターと仮想ネットワークを接続する（しなくても構いません。その場合は独立した単独ネットワークして使います）
* セキュリティグループの作成（今回の演習環境はあらかじめ open-all というセキュリティグループが作成されているのでスキップします）
* キーペアの作成（今回の演習環境はあらかじめ my-key というキーペアが作成されているのでスキップします）
* 仮想マシンの作成
* Floating IP の割り当て

Webのダッシュボードから、Network → Network Topology を開いておくと、変化がわかりやすいくなります。

#### 仮想ルーターの作成

自分用の仮想ルーターを作成し、外部ネットワーク（public/192.168.99.0/24）と接続します。

In [10]:
neutron router-create Ext-Router

Created a new router:
+-------------------------+--------------------------------------+
| Field                   | Value                                |
+-------------------------+--------------------------------------+
| admin_state_up          | True                                 |
| availability_zone_hints |                                      |
| availability_zones      |                                      |
| created_at              | 2017-06-09T02:46:20Z                 |
| description             |                                      |
| external_gateway_info   |                                      |
| flavor_id               |                                      |
| id                      | e22b32e2-7b77-4a1d-880b-8b659339ec67 |
| name                    | Ext-Router                           |
| project_id              | 44be7b065a154e33aacbe7dbca18c6be     |
| revision_number         | 3                                    |
| routes                  |             

In [11]:
neutron router-list

+--------------------------------------+------------+-----------------------+
| id                                   | name       | external_gateway_info |
+--------------------------------------+------------+-----------------------+
| e22b32e2-7b77-4a1d-880b-8b659339ec67 | Ext-Router | null                  |
+--------------------------------------+------------+-----------------------+


In [12]:
neutron router-gateway-set Ext-Router public

Set gateway for router Ext-Router


In [13]:
neutron router-list

+--------------------------------+------------+--------------------------------+
| id                             | name       | external_gateway_info          |
+--------------------------------+------------+--------------------------------+
| e22b32e2-7b77-4a1d-880b-       | Ext-Router | {"network_id": "17e4bba8-2a15  |
| 8b659339ec67                   |            | -46fb-ab54-fb463aee503c",      |
|                                |            | "enable_snat": true,           |
|                                |            | "external_fixed_ips":          |
|                                |            | [{"subnet_id": "a271b0a1-c912  |
|                                |            | -492c-858e-d5bd41888246",      |
|                                |            | "ip_address":                  |
|                                |            | "192.168.99.208"}]}            |
+--------------------------------+------------+--------------------------------+


#### 仮想ネットワークの作成
仮想ネットワークを作成して、この仮想ルーターとの接続を行います。この操作で、仮想ネットワークと public ネットワークが接続されます。

In [14]:
neutron net-create work-net

Created a new network:
+-------------------------+--------------------------------------+
| Field                   | Value                                |
+-------------------------+--------------------------------------+
| admin_state_up          | True                                 |
| availability_zone_hints |                                      |
| availability_zones      |                                      |
| created_at              | 2017-06-09T02:46:38Z                 |
| description             |                                      |
| id                      | 318b7297-2c0e-4bc0-9604-70d85837647f |
| ipv4_address_scope      |                                      |
| ipv6_address_scope      |                                      |
| mtu                     | 1450                                 |
| name                    | work-net                             |
| project_id              | 44be7b065a154e33aacbe7dbca18c6be     |
| revision_number         | 2          

In [15]:
neutron net-list

+---------------------------------+----------+---------------------------------+
| id                              | name     | subnets                         |
+---------------------------------+----------+---------------------------------+
| 17e4bba8-2a15-46fb-             | public   | a271b0a1-c912-492c-858e-        |
| ab54-fb463aee503c               |          | d5bd41888246                    |
| 318b7297-2c0e-                  | work-net |                                 |
| 4bc0-9604-70d85837647f          |          |                                 |
+---------------------------------+----------+---------------------------------+


In [16]:
neutron subnet-create --ip-version 4 --gateway 172.16.11.254 --name work-subnet work-net 172.16.11.0/24

Created a new subnet:
+-------------------+--------------------------------------------------+
| Field             | Value                                            |
+-------------------+--------------------------------------------------+
| allocation_pools  | {"start": "172.16.11.1", "end": "172.16.11.253"} |
| cidr              | 172.16.11.0/24                                   |
| created_at        | 2017-06-09T02:46:48Z                             |
| description       |                                                  |
| dns_nameservers   |                                                  |
| enable_dhcp       | True                                             |
| gateway_ip        | 172.16.11.254                                    |
| host_routes       |                                                  |
| id                | 3f782ee7-faf5-460f-852f-dcfc4287c737             |
| ip_version        | 4                                                |
| ipv6_address_mode |        

In [17]:
neutron net-list

+---------------------------------+----------+---------------------------------+
| id                              | name     | subnets                         |
+---------------------------------+----------+---------------------------------+
| 17e4bba8-2a15-46fb-             | public   | a271b0a1-c912-492c-858e-        |
| ab54-fb463aee503c               |          | d5bd41888246                    |
| 318b7297-2c0e-                  | work-net | 3f782ee7-faf5-460f-852f-        |
| 4bc0-9604-70d85837647f          |          | dcfc4287c737 172.16.11.0/24     |
+---------------------------------+----------+---------------------------------+


In [18]:
neutron router-interface-add Ext-Router work-subnet

Added interface ac501e70-35a9-45f8-8233-804c5859ee91 to router Ext-Router.


#### 仮想マシンを起動します。
この時、仮想マシンは先程作成した仮想ネットワークへ接続を行います。
Cirrosというテスト用のOSイメージを使って仮想マシンを作成します。この仮想マシンは非常にコンパクトでこのようなテストや演習に最適です。
本番環境で利用する際には、CentOSやRHELのイメージを登録しておき、そのイメージを利用します。

In [19]:
function get_uuid () { cat - | grep " id " | awk '{print $4}'; }
export MY_WORK_NET=`neutron net-show work-net | get_uuid`

nova boot --flavor m1.tiny --image "cirros-0.3.5" \
--key-name my-key --security-groups open-all \
--nic net-id=${MY_WORK_NET} \
test-vm-1

+--------------------------------------+-----------------------------------------------------+
| Property                             | Value                                               |
+--------------------------------------+-----------------------------------------------------+
| OS-DCF:diskConfig                    | MANUAL                                              |
| OS-EXT-AZ:availability_zone          |                                                     |
| OS-EXT-STS:power_state               | 0                                                   |
| OS-EXT-STS:task_state                | scheduling                                          |
| OS-EXT-STS:vm_state                  | building                                            |
| OS-SRV-USG:launched_at               | -                                                   |
| OS-SRV-USG:terminated_at             | -                                                   |
| accessIPv4                           |          

仮想マシンの起動を確認します。

In [20]:
nova list

+--------------------------------------+-----------+--------+------------+-------------+----------------------+
| ID                                   | Name      | Status | Task State | Power State | Networks             |
+--------------------------------------+-----------+--------+------------+-------------+----------------------+
| 5c16c5b4-7362-493a-9c61-2b9b423e2e7c | test-vm-1 | ACTIVE | -          | Running     | work-net=172.16.11.8 |
+--------------------------------------+-----------+--------+------------+-------------+----------------------+


ACTIVE状態になるまで待機します。この時、仮想マシンに割り当てられたIPアドレスをメモしておきます。

In [21]:
nova list

+--------------------------------------+-----------+--------+------------+-------------+----------------------+
| ID                                   | Name      | Status | Task State | Power State | Networks             |
+--------------------------------------+-----------+--------+------------+-------------+----------------------+
| 5c16c5b4-7362-493a-9c61-2b9b423e2e7c | test-vm-1 | ACTIVE | -          | Running     | work-net=172.16.11.8 |
+--------------------------------------+-----------+--------+------------+-------------+----------------------+


#### Floating IPの割り当て
作成した仮想マシンは自分で作成した内部ネットワークに接続されており、仮想マシン→外部 の接続は可能ですが、外部→仮想マシンの接続は行なえません。
Floating IPを仮想マシンに割り当てて、外部→仮想マシンの接続が行えるようにします。

Floating IPとは、外部ネットワーク（ここではpublic）のIPアドレスを仮想マシンへNATするための機能です。

まず、管理者が設定している public ネットワークから、Floating IPを確保します。

In [22]:
neutron floatingip-create public

Created a new floatingip:
+---------------------+--------------------------------------+
| Field               | Value                                |
+---------------------+--------------------------------------+
| created_at          | 2017-06-09T02:47:36Z                 |
| description         |                                      |
| fixed_ip_address    |                                      |
| floating_ip_address | 192.168.99.205                       |
| floating_network_id | 17e4bba8-2a15-46fb-ab54-fb463aee503c |
| id                  | 741bdca1-cd81-4282-aaad-052b93fcfed9 |
| port_id             |                                      |
| project_id          | 44be7b065a154e33aacbe7dbca18c6be     |
| revision_number     | 1                                    |
| router_id           |                                      |
| status              | DOWN                                 |
| tenant_id           | 44be7b065a154e33aacbe7dbca18c6be     |
| updated_at          | 2017-

In [23]:
neutron floatingip-list

+------------------------+------------------+---------------------+---------+
| id                     | fixed_ip_address | floating_ip_address | port_id |
+------------------------+------------------+---------------------+---------+
| 741bdca1-cd81-4282     |                  | 192.168.99.205      |         |
| -aaad-052b93fcfed9     |                  |                     |         |
+------------------------+------------------+---------------------+---------+


確保されたIPアドレスを、変数 FIP_UUID へ格納しておきます（後で使います）

In [24]:
FIP_UUID=`neutron floatingip-list -f value -c id --floating_ip_address=192.168.99.205`
echo $FIP_UUID

741bdca1-cd81-4282-aaad-052b93fcfed9


作成した仮想サーバーの論理ポートを調べます。Floating IPの割り当てには、この論理ポートに対して行うためです。

In [25]:
neutron port-list

+-----------------------+------+-------------------+-----------------------+
| id                    | name | mac_address       | fixed_ips             |
+-----------------------+------+-------------------+-----------------------+
| 4ed2c5e9-b018-4bd3-ad |      | fa:16:3e:f0:31:d5 | {"subnet_id":         |
| 25-46a42d3bfadb       |      |                   | "3f782ee7-faf5-460f-  |
|                       |      |                   | 852f-dcfc4287c737",   |
|                       |      |                   | "ip_address":         |
|                       |      |                   | "172.16.11.1"}        |
| ac501e70-35a9-45f8-82 |      | fa:16:3e:74:26:1c | {"subnet_id":         |
| 33-804c5859ee91       |      |                   | "3f782ee7-faf5-460f-  |
|                       |      |                   | 852f-dcfc4287c737",   |
|                       |      |                   | "ip_address":         |
|                       |      |                   | "172.16.11.254"}      |

In [26]:
PORT_UUID=`neutron port-list -f value -c id --fixed_ips ip_address=172.16.11.8`
echo $PORT_UUID

b8757c26-a59f-4927-a3d8-ab5bf8bbfe99


コマンドの引数として、 **Floating IPのUUID** **PortのUUID** の順で指定します

In [27]:
neutron floatingip-associate $FIP_UUID $PORT_UUID

Associated floating IP 741bdca1-cd81-4282-aaad-052b93fcfed9


In [28]:
nova list

+--------------------------------------+-----------+--------+------------+-------------+--------------------------------------+
| ID                                   | Name      | Status | Task State | Power State | Networks                             |
+--------------------------------------+-----------+--------+------------+-------------+--------------------------------------+
| 5c16c5b4-7362-493a-9c61-2b9b423e2e7c | test-vm-1 | ACTIVE | -          | Running     | work-net=172.16.11.8, 192.168.99.205 |
+--------------------------------------+-----------+--------+------------+-------------+--------------------------------------+


付与されたFloating IPへSSH接続します。
自分のノートPCのSSHか、仮想アプライアンスから実行してください。

この時に、/home/openstack/my-key.pem がログインするための秘密鍵になります（あらかじめ今回の演習用に作成して配置したものになります）

In [29]:
ssh -oStrictHostKeyChecking=no -i ~/my-key.pem cirros@192.168.99.205 hostname

test-vm-1


#### 後片付け
仮想マシンを削除し、割り当てたFloating IPも解除しておきます。

In [30]:
nova delete test-vm-1

Request to delete server test-vm-1 has been accepted.


In [31]:
nova list

+----+------+--------+------------+-------------+----------+
| ID | Name | Status | Task State | Power State | Networks |
+----+------+--------+------------+-------------+----------+
+----+------+--------+------------+-------------+----------+


以下で指定するUUIDは、上で作成したFloating IPのUUIDです。

In [32]:
neutron floatingip-delete $FIP_UUID

Deleted floatingip(s): 741bdca1-cd81-4282-aaad-052b93fcfed9


In [33]:
neutron floatingip-list

作成した仮想ルーター、仮想ネットワークはこの後のハンズオン用に残しておきます。